In [5]:
import os
import json
import random
import sys

# 플레이어 클래스
class Player:
    def __init__(self, name):
        self.name = name
        self.health = 100
        self.mental = 100
        self.money = 50
        self.inventory = []

    def update_stats(self, effect):
        self.health += effect.get("health", 0)
        self.mental += effect.get("mental", 0)
        self.money += effect.get("money", 0)
        item = effect.get("item")
        if item:
            self.inventory.append(item)
            print(f"\n아이템 '{item}'을(를) 획득했습니다.")
        self.print_status()

    def consume_item(self, item_name):
        if item_name in self.inventory:
            self.inventory.remove(item_name)
            print(f"\n아이템 '{item_name}'을(를) 사용하여 소모했습니다.")
        else:
            print(f"\n아이템 '{item_name}'이(가) 인벤토리에 없습니다.")

    def is_game_over(self):
        return self.health <= 0 or self.mental <= 0

    def print_status(self):
        print(f"\n현재 상태 - 체력: {self.health}, 멘탈: {self.mental}, 돈: {self.money}")
        print(f"인벤토리: {self.inventory}\n")


# JSON 파일 로드 함수
def load_encounter(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        return json.load(f)


# 선택지 조건 확인 함수
def is_choice_available(player, requirements):
    if not requirements:
        return True
    if "min_health" in requirements and player.health < requirements["min_health"]:
        return False
    if "min_mental" in requirements and player.mental < requirements["min_mental"]:
        return False
    if "min_money" in requirements and player.money < requirements["min_money"]:
        return False
    if "requires_item" in requirements:
        if requirements["requires_item"] not in player.inventory:
            return False
    return True

# 게임 즉시 종료 함수수
def get_input(prompt):
    user_in = input(prompt)
    if user_in.lower() == 'x':
        print("x 입력으로 게임을 종료합니다.")
        sys.exit(0)
    return user_in

# 단일 인카운터 진행 함수 (단계가 없는 경우)
def run_encounter(player, encounter_data):
    print("\n" + "="*40)
    print("스토리:")
    print(encounter_data.get("story", ""))
    print("="*40)

    available_choices = []
    for idx, choice in enumerate(encounter_data["choices"]):
        req = choice.get("requirements", {})
        if is_choice_available(player, req):
            available_choices.append((idx, choice))
            print(f"{idx + 1}. {choice['description']}")
        else:
            print(f"{idx + 1}. {choice['description']} (선택 불가)")

    while True:
        try:
            choice_input = int(get_input("번호를 선택하세요: ")) - 1
            if any(choice_input == idx for idx, _ in available_choices):
                break
            else:
                print("해당 선택지는 사용할 수 없습니다. 다시 선택해주세요.")
        except ValueError:
            print("숫자를 입력해주세요.")

    chosen_choice = encounter_data["choices"][choice_input]
    print("\n" + chosen_choice["player_response"])

    if "consume_item" in chosen_choice:
        player.consume_item(chosen_choice["consume_item"])

    player.update_stats(chosen_choice["effect"])


# 연속된 스토리(단계)를 가진 인카운터 진행 함수
def run_continuous_encounter(player, encounter_data):
    stages = encounter_data.get("stages", [])
    for stage in stages:
        print("\n" + "="*40)
        print("스토리:")
        print(stage.get("story", ""))
        print("="*40)

        available_choices = []
        for idx, choice in enumerate(stage["choices"]):
            req = choice.get("requirements", {})
            if is_choice_available(player, req):
                available_choices.append((idx, choice))
                print(f"{idx + 1}. {choice['description']}")
            else:
                print(f"{idx + 1}. {choice['description']} (선택 불가)")

        if not available_choices:
            print("사용 가능한 선택지가 없습니다. 이 단계를 건너뜁니다.")
            continue

        while True:
            try:
                choice_input = int(get_input("번호를 선택하세요: ")) - 1
                if any(choice_input == idx for idx, _ in available_choices):
                    break
                else:
                    print("해당 선택지는 사용할 수 없습니다. 다시 선택해주세요.")
            except ValueError:
                print("숫자를 입력해주세요.")

        chosen_choice = stage["choices"][choice_input]
        print("\n" + chosen_choice["player_response"])

        if "consume_item" in chosen_choice:
            player.consume_item(chosen_choice["consume_item"])

        player.update_stats(chosen_choice["effect"])

        if player.is_game_over():
            print("체력이나 멘탈이 0 이하가 되어 게임 오버입니다!")
            return  # 게임 종료

In [6]:
# Introduction (캐릭터 생성 및 시작 아이템 지급)

def introduction(player):
    print("\n=== 캐릭터 생성 및 게임 소개 ===")
    print(f"{player.name}님, 게임에 오신 것을 환영합니다!")
    starting_items = ["행운의 부적", "낡은 지도", "비밀의 열쇠"]
    item = random.choice(starting_items)
    player.inventory.append(item)
    print(f"시작 아이템으로 '{item}'을(를) 획득했습니다.")
    player.print_status()


# 인카운터 파일 목록 로드 함수
def load_encounter_list(directory):
    files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".json")]
    return files

In [7]:
# 게임 루프 (사이클 반복: 랜덤 인카운터 3회 후 남은 메인 인카운터 등장)

def game_loop(player):
    random_encounter_dir = "./random_encounters"
    main_encounter_dir = "./main_encounters"
    random_encounters = load_encounter_list(random_encounter_dir)
    available_main_encounters = load_encounter_list(main_encounter_dir)  # 메인 인카운터는 한 번씩만 등장
    previous_random_encounter = None
    MIN_RANDOM_ENCOUNTERS = 3
    cycle = 1

    while True:
        random_count = 0
        # 최소 MIN_RANDOM_ENCOUNTERS 만큼 랜덤 인카운터 진행
        while random_count < MIN_RANDOM_ENCOUNTERS:
            encounter_file = random.choice(random_encounters)
            # 직전 인카운터와 동일하면 건너뜀 (단, 파일이 2개 이상이면)
            if encounter_file == previous_random_encounter and len(random_encounters) > 1:
                continue
            encounter_data = load_encounter(encounter_file)
            run_encounter(player, encounter_data)
            if player.is_game_over():
                return
            previous_random_encounter = encounter_file
            random_count += 1

        # 남은 메인 인카운터가 있다면 등장
        if available_main_encounters:
            main_encounter_file = random.choice(available_main_encounters)
            available_main_encounters.remove(main_encounter_file)
            main_encounter_data = load_encounter(main_encounter_file)
            if "stages" in main_encounter_data:
                run_continuous_encounter(player, main_encounter_data)
            else:
                run_encounter(player, main_encounter_data)
            if player.is_game_over():
                return
        else:
            print("더 이상 등장할 메인 인카운터가 없습니다. 랜덤 인카운터만 진행합니다.")

In [8]:
# 3. 메인 게임 루프 (재시작 기능 포함)

def main():
    player_name = input("플레이어 이름을 입력하세요: ")
    player = Player(player_name)
    introduction(player)
    game_loop(player)
    print("게임이 종료되었습니다.")

if __name__ == "__main__":
    while True:
        main()
        replay = input("\n게임이 종료되었습니다. 다시 플레이하시겠습니까? (y/n): ")
        if replay.lower() != 'y':
            print("플레이해주셔서 감사합니다. 게임을 종료합니다.")
            break


=== 캐릭터 생성 및 게임 소개 ===
x님, 게임에 오신 것을 환영합니다!
시작 아이템으로 '비밀의 열쇠'을(를) 획득했습니다.

현재 상태 - 체력: 100, 멘탈: 100, 돈: 50
인벤토리: ['비밀의 열쇠']


스토리:
당신은 어두운 동굴 입구를 발견했습니다.
1. 동굴에 들어간다.
2. 동굴을 피한다.

당신은 동굴에 들어가고 보석을 발견합니다.

아이템 '보석'을(를) 획득했습니다.

현재 상태 - 체력: 95, 멘탈: 105, 돈: 50
인벤토리: ['비밀의 열쇠', '보석']


스토리:
당신은 강가를 지나가다가 다리가 무너진 것을 봅니다.
1. 강을 건너려 한다.
2. 다른 길을 찾아본다.
x 입력으로 게임을 종료합니다.


SystemExit: 0

C:\Users\dodam\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
